## Import and Clean Data

In [46]:
import pandas as pd
import matplotlib.pyplot as plt


In [47]:
raw_data = pd.read_csv("../data/Salary.csv")
education_levels = ["high school", "bachelor degree", "masters degree", "phd"]
raw_data

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000.0,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000.0,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000.0,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000.0,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000.0,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
6679,49.0,Female,3,Director of Marketing,20.0,200000.0,UK,Mixed,0
6680,32.0,Male,0,Sales Associate,3.0,50000.0,Australia,Australian,0
6681,30.0,Female,1,Financial Manager,4.0,55000.0,China,Chinese,0
6682,46.0,Male,2,Marketing Manager,14.0,140000.0,China,Korean,0


In [48]:
df = raw_data[["Job Title","Education Level","Years of Experience","Country","Salary"]]
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [49]:
isomapping = {"usa":"us", "china":"cn","australia":"au","uk":"uk","canada":"ca"}
df['country'] = df['country'].str.lower().map(isomapping)

C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_9740\1632140083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['country'] = df['country'].str.lower().map(isomapping)


In [50]:
df['job_title']=df['job_title'].str.lower()

C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_9740\1849090496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_title']=df['job_title'].str.lower()


In [51]:
df['education_level']

0       1
1       2
2       3
3       1
4       2
       ..
6679    3
6680    0
6681    1
6682    2
6683    0
Name: education_level, Length: 6684, dtype: int64

In [52]:
sampled_df = df.sample(n=1000, replace=True, random_state=42)


## Setting up Lin Reg Model with word embeddings

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sentence_transformers import SentenceTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [54]:
features = ["job_title","education_level", "years_of_experience", "country"]

In [55]:
X = sampled_df[features]
y = sampled_df['salary']

In [56]:
X2 = df[features]
y2 = df["salary"]
X_train2, X_test2, y_test2, y_test2 = train_test_split(X2, y2, test_size=0.33, random_state=42)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [58]:
def evaluate_regression_metrics(y_true, y_pred):
    """
    Evaluate regression metrics: Mean Absolute Error, Mean Squared Error, and R-squared.

    Parameters:
    y_true (array-like): Actual target values.
    y_pred (array-like): Predicted target values.

    Returns:
    dict: A dictionary containing MAE, MSE, and R-squared.
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return {
        'Mean Absolute Error': mae,
        'Mean Squared Error': mse,
        'R-squared': r2
    }

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin


class SentenceTransformerEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="nomic-ai/nomic-embed-text-v1"):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)
        self.model_name = model_name


    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Encode the text data into embeddings
        return self.model.encode(X.tolist())

In [71]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]
embeddings = model.encode(sentences)
print(embeddings.shape)

C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wasadmin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\huggingface_hub

OSError: [Errno 28] No space left on device

In [60]:
testencoder = SentenceTransformerEncoder()


ValueError: nomic-ai/nomic-bert-2048 You can inspect the repository content at https://hf.co/nomic-ai/nomic-embed-text-v1.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('job_title', SentenceTransformerEncoder(), 'job_title'),  # Encode job_title
        ('years_of_experience', StandardScaler(), ['years_of_experience']),  # Scale years_of_experience
        ('country', OneHotEncoder(handle_unknown='ignore'), ['country'])  # One-hot encode country
    ]
)

word_processing_pipeline = Pipeline([
    ('preprocessor', preprocessor),  
    (('linreg'), LinearRegression())
])

<All keys matched successfully>


In [ ]:
word_processing_pipeline.fit(X_train, y_train)

<All keys matched successfully>


,steps,"[('preprocessor', ...), ('linreg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('job_title', ...), ('years_of_experience', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
word_processing_pipeline.score(X_test, y_test)

0.8060153286422945

In [ ]:
evaluate_regression_metrics(word_processing_pipeline.predict(X_test), y_test)

{'Mean Absolute Error': 17519.27089207211,
 'Mean Squared Error': 552511509.8639787,
 'R-squared': 0.7880120093660397}

## Exporting the Model

In [ ]:
import pickle as pkl

In [ ]:
filepath = "../models/word_embedding_model.pkl"
with open(filepath, "wb") as f:
    pkl.dump(word_processing_pipeline, f)

OSError: [Errno 28] No space left on device